***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-38d1  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  7 07:40 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket324955145' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()

In [8]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [9]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [10]:
from inverted_index_gcp import InvertedIndex

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

In [12]:
# path = '/home/dataproc'
for blob in client.list_blobs(bucket_name):
    if blob.name.endswith('.pkl'):
        file_name = blob.name[blob.name.rfind('/')+1:]
        blob.download_to_filename(file_name)

In [13]:
with open('postings_gcp_title_inverted_title_v1.pkl', 'rb') as file:
    inverted_title = pickle.load(file)
    
with open('postings_gcp_text_inverted_text_v1.pkl', 'rb') as file:
    inverted_text = pickle.load(file)

In [14]:
from pyspark.sql.types import MapType, StringType, IntegerType 
from pyspark.sql.functions import create_map, lit

In [15]:
def calculate_document_length(postings):
    return postings.map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda a, b: a + b)

In [16]:
# word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
# word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

# # calculate document lengths:
# title_doc_length = calculate_document_length(word_counts_title)
# text_doc_length = calculate_document_length(word_counts_text)

# # title_length = title_doc_length.collectAsMap()
# # document_length = text_doc_length.collectAsMap()

# total_title_length = title_doc_length.map(lambda x: x[1]).sum()
# total_title_docs = title_doc_length.count()

# # Calculate total length and total number of documents for texts:
# total_text_length = text_doc_length.map(lambda x: x[1]).sum()
# total_text_docs = text_doc_length.count()

# # Calculate avdl:
# average_title_length = total_title_length / total_title_docs
# average_text_length = total_text_length / total_text_docs

In [ ]:
text_doc_len = inverted_text.doc_len.copy()
title_doc_len = inverted_title.doc_len.copy()

# an attempt at making the these into dataframes
# mapping_expr = create_map("my_map_column", [lit(k) for k, v in text_doc_len.items()])
# df_text_doc_len = df.withColumn("value", mapping_expr[col("key")])

In [ ]:
# text_idf = sc.broadcast(inverted_text.idf)
# title_idf = sc.broadcast(inverted_title.idf)

text_idf = inverted_text.idf
title_idf = inverted_title.idf

In [ ]:
# text_idf_bm25 = sc.broadcast(inverted_text.get_idf_bm25())
# title_idf_bm25 = sc.broadcast(inverted_title.get_idf_bm25())

text_idf_bm25 = inverted_text.get_idf_bm25()
title_idf_bm25 = inverted_title.get_idf_bm25()

In [ ]:
# takes like a minute so run only when the normalized pageranks are needed:
# =========================================================================
text_pagerank = inverted_text.pagerank_normalized
title_pagerank = inverted_title.pagerank_normalized

In [ ]:
# text_avdl = sc.broadcast(inverted_text.get_avdl())
# title_avdl = sc.broadcast(inverted_title.get_avdl())

text_avdl = inverted_text.get_avdl()
title_avdl = inverted_title.get_avdl()

In [ ]:
# check if we're on the right pickle
print(len(text_doc_len))
print(len(title_idf_bm25))
print(title_avdl)
print(len(title_pagerank))

In [ ]:
k1 = 0.1
k3 = 0
b = 0.75

In [ ]:
def retrieve_posting_list(query_word: str):
#     text_pl = inverted_text.read_a_posting_list(base_dir='.', w=query_word, bucket_name=bucket_name)
    title_pl = inverted_title.read_a_posting_list(base_dir='.', w=query_word, bucket_name=bucket_name)
    # return text_pl, title_pl
    return title_pl

# def combine_title_and_text(token, pls):
#     text_pl, title_pl = pls
#     text_pl = sc.parallelize(text_pl)
#     text_pl_normalized = text_pl.map(lambda x: (x[0], ((x[1] / text_doc_len.get(x[0], float('inf'))))))
#     # text_pl_normalized = text_pl.map(lambda x: (x[0], ((x[1] / text_doc_len.get(x[0], float('inf'))) * text_idf[token])))
#     # text_pl_normalized = text_pl.map(lambda x: (x[0], ((x[1] / text_doc_len.get(x[0], float('inf'))) * text_idf[token] * text_pagerank.get(x[0], 0.001))))
#     title_pl = sc.parallelize(title_pl)
#     title_pl_normalized = title_pl.map(lambda x: (x[0], ((x[1] / title_doc_len.get(x[0], float('inf'))))))
#     # title_pl_normalized = title_pl.map(lambda x: (x[0], ((x[1] / title_doc_len.get(x[0], float('inf'))) * title_idf[token])))
#     # title_pl_normalized = title_pl.map(lambda x: (x[0], ((x[1] / title_doc_len.get(x[0], float('inf'))) * title_idf[token] * title_pagerank.get(x[0], 0.001))))
#     combined_rdd = text_pl_normalized.union(title_pl_normalized)
#     return combined_rdd

def query(query: str):
    tokens = [token.group() for token in RE_WORD.finditer(query.lower())]
    tokens = [token for token in tokens if token not in all_stopwords]
    tokens = sc.parallelize(tokens)
    
    posting_lists = tokens.map(lambda x: (x, retrieve_posting_list(x))).flatMapValues(lambda x: x)
#     bm_25 on title:
    bm_25 = posting_lists.map(lambda x: (x[1][0], ((x[1][1] * (k1 + 1)) / (1 - b + (b * title_doc_len[x[1][0]] / title_avdl))) * title_idf_bm25[x[0]]))
    summed_rdd = bm_25.reduceByKey(lambda x, y: x + y)
    pr = summed_rdd.map(lambda x: (x[0], title_pagerank[x[0]]*x[1]))
    sorted_rdd = pr.sortBy(lambda pair: pair[1], ascending=False)
    sorted_rdd_title = sorted_rdd.zipWithIndex().filter(lambda x: x[1] < 1000).map(lambda x: x[0])
    
    print('hi')
    return (sorted_rdd_title.take(30))
    
# #     bm_25 on text:
#     bm_25 = sorted_rdd_title.map(lambda x: (x[1][0], ((x[1][1] * (k1 + 1)) / (1 - b + (b * text_doc_len[x[1][0]] / text_avdl))) * text_idf_bm25[x[0]]))
#     summed_rdd = bm_25.reduceByKey(lambda x, y: x + y)
#     pr = summed_rdd.map(lambda x: (x[0], text_pagerank[x[0]]*x[1]))
#     sorted_rdd = pr.sortBy(lambda pair: pair[1], ascending=False)

    # posting_lists = [retrieve_posting_list(token) for token in tokens]
    # combined_posting_lists = [combine_title_and_text(*pl) for pl in posting_lists]
    # combined_rdd = sc.union(combined_posting_lists)
    # combined_rdd = sc.union(posting_lists)
    # summed_rdd = combined_rdd.reduceByKey(lambda x, y: x + y)
#     pr = summed_rdd.map(lambda x: (x[0], text_pagerank[x[0]]*x[1]))
#     sorted_rdd = pr.sortBy(lambda pair: pair[1], ascending=False)

#     w/ out pr:
#     sorted_rdd = summed_rdd.sortBy(lambda pair: pair[1], ascending=False)

    
#     return (sorted_rdd.take(30))

In [ ]:
query('orchard')

In [ ]:
# query('the Austro-Hungarian ultimatum to Serbia on 23 July')

In [ ]:
# query('berlin wall')

In [ ]:
# plplp = sc.parallelize([token for token in ['giọt', 'dunsby']]).map(lambda x: (x, retrieve_posting_list(x)))

In [ ]:
# plplp.collect()

In [ ]:
# plplp.flatMapValues(lambda x: x).map(lambda x: (x[1][0], ((x[1][1] / text_doc_len.get(x[1][0], float('inf'))) * text_idf[x[0]] * text_pagerank.get(x[1][0], 0.001)))).collect()

In [ ]:
# inverted_text.df